In [1]:
import sys
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
from matplotlib.colors import LinearSegmentedColormap
# from matplotlib import colormaps
import matplotlib.gridspec as gridspec
import pandas as pd
from config import predpath, survpath, phedefpath, phecodespath, phecatpath, trainpath, temppath, outputpath
sys.path.append(trainpath)
# import survutil


In [ ]:
model = torch.load("../../results/weights/pred/610_3model", map_location=torch.device('cpu'))
corrprediction=model.outlayer.weight.data.to('cpu').numpy()
model_surv = torch.load("../../results/weights/surv/610_0model", map_location=torch.device('cpu'))
corrsurvival=model_surv.outlayer.weight.cpu().detach().numpy()

In [ ]:
random.seed(0)
np.random.seed(0)
X = corrprediction
tsne = TSNE()
X_tsne_prediction = tsne.fit_transform(X)

In [31]:
phecode_col = pd.read_csv("../../data/phecode.csv")
phecat = np.load(phecatpath, allow_pickle=True)[0]

In [14]:
# Convert dictionary to two-column DataFrame
rows = []
for category, indices in phecat.items():
    for index in indices:
        rows.append([index, category])

df = pd.DataFrame(rows, columns=["Row_Number", "Category"])

# Save DataFrame to CSV
output_path = "./results/cache/multimorbidity/phecat_rows_and_categories.csv"
df.to_csv(output_path, index=False)

In [32]:
# Save
phecode_col = phecode_col["Phecode"].astype(str).reset_index(drop=True)

# Convert to DataFrame with updated column names
corrprediction_df = pd.DataFrame(corrprediction, index=phecode_col)
corrsurvival_df = pd.DataFrame(corrsurvival, index=phecode_col)
X_tsne_prediction_df = pd.DataFrame(X_tsne_prediction, columns=["TSNE_1", "TSNE_2"], index=phecode_col)

# Save as CSV
corrprediction_df.to_csv("./results/cache/multimorbidity/multimorbidity.csv", index_label="Phecode")
corrsurvival_df.to_csv("./results/cache/multimorbidity/multimorbidity_surv.csv", index_label="Phecode")
X_tsne_prediction_df.to_csv("./results/cache/multimorbidity/X_tsne_prediction.csv", index_label="Phecode")